# Иерархия холдинга
Пример расчета с учетом иерархии холдинга.

## Описание задачи

Рассчитаем аудиторию холдинга с разбивкой на все входящие в него медиа объекты.
Построение иерархии возможно для всех уровней медиа объектов в рамках одной ветки. 

Иерархия:

- Holding-Project-Section-Subsection
- Network-Network Section-Network subsection
- AdAgency-Brand-Subbrand-Position


Общие параметры:
- Период: 1-30 июня 2020 года
- География: Россия 100 000+
- Население: 12+
- Фильтр по типу использования: нет, считаем по всем (Web Desktop, Web Mobile, App Online, App Offline).

Статистики:
- Unweighted Reach (UnwReach)
- Reach


## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к API Responsum и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from pathlib import Path
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from IPython.display import JSON
from bokeh.models import HoverTool
from bokeh.layouts import gridplot
import logging

# from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from mediascope_api.core import net as msnet
from mediascope_api.responsum import catalogs as rc
from mediascope_api.responsum import tasks as rt

logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# pd.set_option("display.max_rows", 200)
pd.set_option("display.max_colwidth", None)
# pd.set_option("display.precision", 6)
output_notebook()

### Общие параметры для заданий

Для начала зададим общие параметры

In [ ]:
# задаем параметры
# выбираем тип установки mobile, т.к. в расчетах участвуют все типы устройств (Web Desktop, Web Mobile, App Online, App Offline)
facility = 'mobile' # возможные значения: 'desktop', 'mobile', 'desktop_pre'

# создаем объекты для работы с каталогами и заданиями,
# а так же загружаем каталоги
rcats = rc.ResponsumCats(facility)
rtask = rt.ResponsumTask(facility)

# задаем период расчета
date_from = '2020-06-01'
date_to = '2020-06-30'

# задаем типы пользования Интернетом
usetypes = rcats.get_usetype('all')

# проверяем, что значения параметров установлены верно
rtask.save_report_info(facility, date_from, date_to, usetypes)
rtask.show_report_info()
print(f"Объектов в media-каталоге: {rcats.holdings.shape[0]}")

### Получим ID холдинга
Для построения отчета нам необходимо получить идентификатор холдинга __Google Sites__, для этого воспользуемся ноутбуком, в котором приведены примеры работы с [каталогами](catalogs.ipynb):

- Google Projects holding_id = 88149

## Задания
Перейдем к формированию заданий.

### Задание №1. Построение иерархии


In [ ]:
# задаем название для отображения в DataFrame
project_name = 'hierarchy'

# задаем медиа фильтр, в нашем случае это ID холдинга Google Sites
media_filter = "holding = 88149"

# задаем фильтр, нас интересуют города России с населением 100 тыс. чел. и больше
demo_filter = "CITY_TYPE2 != 4"

# задаем список статистик для расчета
statistics = ["UnwReach", "Reach"]

# указываем порядок разбивки, 
# в оси media перечислим все уровни, которые хотим получить в виде иерархии
structure =  {
    "usetype": False,
    "media": ["holding", "site", "section", "subsection" ]
  }
 
# формируем из заданных параметров задание для Responsum в формате JSON
task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)

# отправляем задание на расчет и ждем выполнения
task_audience = rtask.wait_task(rtask.send_audience_task(task_json))

# получаем результат
df_hierarchy = rtask.result2hierarchy(rtask.get_result(task_audience), project_name=project_name)
df_hierarchy

## Экспорт в Excel

In [ ]:
writer = pd.ExcelWriter(rtask.get_excel_filename('hierarсhy'))
df_info = rtask.get_report_info()
df_hierarchy.to_excel(writer, 'Report', index=False)
df_info.to_excel(writer, 'Info', index=False)
writer.save()